In [1]:
import numpy as np
from load_data import load_data
from search_space import test_create_search_space

Using TensorFlow backend.


In [2]:
train, valid = load_data()
train_x = train[0]
train_y = train[1]
valid_x = valid[0]
valid_y = valid[1]
train_x = [np.array(train_x[0].todense()), np.array(train_x[1].todense())]
valid_x = [np.array(valid_x[0].todense()), np.array(valid_x[1].todense())]
print(valid_x[0].shape)
print(valid_x[1].shape)

Loading cora dataset
Pre-processing node features
================ Load Data ================
A_train shape: (2031, 2708)
X_train shape: (2031, 1433)
y_train shape: (2031, 7)
A_valid shape: (677, 2708)
X_valid shape: (677, 1433)
y_valid shape: (677, 7)


(677, 1433)
(677, 2708)


In [3]:
model = test_create_search_space()

This search_space needs 2 choices to generate a neural network.
Input shape: [TensorShape([Dimension(None), Dimension(1433)]), TensorShape([Dimension(None), Dimension(2708)])]

Input shape: [TensorShape([Dimension(None), Dimension(48)]), TensorShape([Dimension(None), Dimension(2708)])]
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_0 (InputLayer)            [(None, 1433)]       0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2708)]       0                                            
___________________________________________________________________________________

In [4]:
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])

In [5]:
model.fit(train_x, train_y, validation_data=(valid_x, valid_y), epochs=20, batch_size=2031)

Train on 2031 samples, validate on 677 samples
Epoch 1/20


InvalidArgumentError: Matrix size-incompatible: In[0]: [2031,2708], In[1]: [2031,48]
	 [[{{node graph_conv/MatMul_1}}]]

In [1]:
"""
This example implements the experiments on citation networks from the paper:
Semi-Supervised Classification with Graph Convolutional Networks (https://arxiv.org/abs/1609.02907)
Thomas N. Kipf, Max Welling
"""
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

from spektral.datasets import citation
from spektral.layers import GraphConv
from spektral.utils import localpooling_filter

Using TensorFlow backend.


In [2]:
# Load data
dataset = 'cora'
A, X, y, train_mask, val_mask, test_mask = citation.load_data(dataset)

# Parameters
channels = 16           # Number of channels in the first layer
N = X.shape[0]          # Number of nodes in the graph
F = X.shape[1]          # Original feature dimensionality
n_classes = y.shape[1]  # Number of classes
dropout = 0.5           # Dropout rate applied to the features
l2_reg = 5e-4           # Regularization rate for l2
learning_rate = 1e-2    # Learning rate for SGD
epochs = 20000          # Number of training epochs
es_patience = 200       # Patience for early stopping

# Preprocessing operations
fltr = localpooling_filter(A)
y.shape

Loading cora dataset
Pre-processing node features


(2708, 7)

In [3]:
# Model definition
X_in = Input(shape=(F, ))
fltr_in = Input((N, ), sparse=True)

In [4]:
dropout_1 = Dropout(dropout)(X_in)
graph_conv_1 = GraphConv(channels,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         use_bias=False)([dropout_1, fltr_in])


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


TypeError: Failed to convert object of type <class 'tuple'> to Tensor. Contents: (Dimension(1433), 16). Consider casting elements to a supported type.

In [ ]:
dropout_2 = Dropout(dropout)(graph_conv_1)
graph_conv_2 = GraphConv(n_classes,
                         activation='softmax',
                         use_bias=False)([dropout_2, fltr_in])

# Build model
model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_2)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

# Train model
validation_data = ([X, fltr], y, val_mask)

In [ ]:
model.fit([X, fltr],
          y,
          sample_weight=train_mask,
          epochs=epochs,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,  # Shuffling data means shuffling the whole graph
          callbacks=[
              EarlyStopping(patience=es_patience,  restore_best_weights=True)
          ])

# Evaluate model
print('Evaluating model.')
eval_results = model.evaluate([X, fltr],
                              y,
                              sample_weight=test_mask,
                              batch_size=N)
print('Done.\n'
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))